# download the dataset from 
https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

# Ungraded Lab: Training with ImageDataGenerator
In this lab, you will build a train a model on the Horses or Humans dataset. This contains over a thousand images of horses and humans with varying poses and filesizes. You will use the ImageDataGenerator class to prepare this dataset so it can be fed to a convolutional neural network.

IMPORTANT NOTE: This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

Run the code below to download the compressed dataset horse-or-human.zip.

In [2]:
import zipfile

In [5]:
#unzip the datasets
local_zip='datasets/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('datasets/horse-or-human')
zip_ref.close()

In [6]:
import os
train_horse_dir=os.path.join('datasets/horse-or-human/horses/')
train_human_dir=os.path.join('datasets/horse-or-human/humans/')

In [10]:
train_horse_names=os.listdir(train_horse_dir)
print(train_horse_names[:10])
train_human_names=os.listdir(train_human_dir)
print(train_human_names[:10])

['horse01-0.png', 'horse01-1.png', 'horse01-2.png', 'horse01-3.png', 'horse01-4.png', 'horse01-5.png', 'horse01-6.png', 'horse01-7.png', 'horse01-8.png', 'horse01-9.png']
['human01-00.png', 'human01-01.png', 'human01-02.png', 'human01-03.png', 'human01-04.png', 'human01-05.png', 'human01-06.png', 'human01-07.png', 'human01-08.png', 'human01-09.png']
